In [1]:
import numpy as np
import pandas as pd

In [2]:
movies=pd.read_csv('TMDB_all_movies.csv', engine='python', on_bad_lines='warn')

In [3]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140352 entries, 0 to 1140351
Data columns (total 28 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   id                       1140352 non-null  int64  
 1   title                    1140339 non-null  object 
 2   vote_average             1140352 non-null  float64
 3   vote_count               1140352 non-null  float64
 4   status                   1140352 non-null  object 
 5   release_date             1022557 non-null  object 
 6   revenue                  1140352 non-null  float64
 7   runtime                  1140352 non-null  float64
 8   budget                   1140352 non-null  float64
 9   imdb_id                  644707 non-null   object 
 10  original_language        1140352 non-null  object 
 11  original_title           1140340 non-null  object 
 12  overview                 964954 non-null   object 
 13  popularity               1140352 non-null 

In [5]:
movies=movies[['id' ,'original_title','overview','genres','cast','director','writers']]

In [6]:
movies.dropna(inplace=True)


In [12]:
movies.head()

,id,original_title,overview,genres,cast,director,writers
0,2,Ariel,"[A, Finnish, man, goes, to, the, city, to, fin...","[Comedy, Drama, Romance, Crime]","[SusannaHaavisto, PenttiAuer, OlliVarja]",[AkiKaurismäki],[AkiKaurismäki]
1,3,Varjoja paratiisissa,"[Nikander,, a, rubbish, collector, and, would-...","[Comedy, Drama, Romance]","[HelmeriPellonpää, UllaKuosmanen, MarinaMartin...",[AkiKaurismäki],[AkiKaurismäki]
2,5,Four Rooms,"[It's, Ted, the, Bellhop's, first, night, on, ...",[Comedy],"[TamlynTomita, UnrulyJulieMcClean, LiliTaylor]","[AlexandreRockwell, RobertRodriguez, QuentinTa...","[AlexandreRockwell, RobertRodriguez, QuentinTa..."
3,6,Judgment Night,"[Four, young, friends,, while, taking, a, shor...","[Action, Crime, Thriller]","[DougWert, AngelaAlvarado, Everlast]",[StephenHopkins],"[LewisColick, JereCunningham]"
6,11,Star Wars,"[Princess, Leia, is, captured, and, held, host...","[Adventure, Action, ScienceFiction]","[DavidProwse, RickMcCallum, MandyMorton]",[GeorgeLucas],[GeorgeLucas]


In [9]:
movies['overview']=movies['overview'].apply(lambda x:x.split())
movies['genres']=movies['genres'].apply(lambda x:x.split(','))
movies['director']=movies['director'].apply(lambda x:x.split(','))
movies['writers']=movies['writers'].apply(lambda x:x.split(','))
movies['cast'] = movies['cast'].apply(lambda x: [c.strip() for c in x.split(',')[:3]])

In [11]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movies['director']=movies['director'].apply(lambda x:[i.replace(' ','') for i in x])
movies['writers']=movies['writers'].apply(lambda x:[i.replace(' ','') for i in x])

In [13]:
movies['tags'] = movies[['overview', 'cast', 'genres', 'director', 'writers']].apply(lambda x: sum([item if isinstance(item, list) else [] for item in x], []), axis=1)

Cleaning the new collected data


In [14]:
new_df=movies[['id','original_title','tags']]

In [15]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

/tmp/ipython-input-487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [16]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

/tmp/ipython-input-4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [17]:
new_df.head()

,id,original_title,tags
0,2,Ariel,a finnish man goes to the city to find a job a...
1,3,Varjoja paratiisissa,"nikander, a rubbish collector and would-be ent..."
2,5,Four Rooms,it's ted the bellhop's first night on the job....
3,6,Judgment Night,"four young friends, while taking a shortcut en..."
6,11,Star Wars,princess leia is captured and held hostage by ...


Removing repeating or similar words


In [18]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [19]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [20]:
new_df['tags']=new_df['tags'].apply(stem)

/tmp/ipython-input-3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


Removing Stop Words


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words="english")

In [22]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
def recommend(movie):
  movie_index = new_df[new_df['title']==movie].index[0]
  distance= similarity[movie_index]
  movies_list=sorted(list(enumerate(distance)),reverse=True,key= lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_df.iloc[i[0]].title)
  return